# Interactive Chat Application with Azure AI Projects

This notebook demonstrates how to build an interactive chat application using Azure AI Projects and OpenAI. The notebook is organized into modular cells that allow you to experiment with different aspects of the chat functionality.

## Prerequisites
- Azure AI Project configured
- Environment variables set in `.env` file
- Required Python packages installed

## 1. Install and Import Required Libraries

First, let's install and import all the necessary libraries for our chat application.

In [1]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    "python-dotenv",
    "azure-identity",
    "azure-ai-projects"
]

# Install packages
for package in required_packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✓ {package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        install_package(package)

Installing python-dotenv...
Installing azure-identity...
Installing azure-ai-projects...


In [2]:
# Import required libraries
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from IPython.display import display, Markdown, clear_output
import json
from datetime import datetime

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## 2. Configuration Setup

Load environment variables and configure the Azure AI Project client.

In [3]:
# Load environment variables
load_dotenv()

# Get configuration settings
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT")

# Validate configuration
if not project_endpoint:
    raise ValueError("PROJECT_ENDPOINT not found in environment variables")
if not model_deployment:
    raise ValueError("MODEL_DEPLOYMENT not found in environment variables")

print(f"✓ Project Endpoint: {project_endpoint}")
print(f"✓ Model Deployment: {model_deployment}")

✓ Project Endpoint: https://gpt40chat-resource.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview
✓ Model Deployment: gpt-4o-mini


## 3. Initialize Azure AI Project Client

Create the Azure AI Project client and get the OpenAI client for chat completions.

In [4]:
# Initialize the project client
try:
    project_client = AIProjectClient(
        credential=DefaultAzureCredential(
            exclude_environment_credential=True,
            exclude_managed_identity_credential=True
        ),
        endpoint=project_endpoint,
    )
    
    # Get a chat client
    openai_client = project_client.get_openai_client(api_version="2024-10-21")
    
    print("✓ Azure AI Project client initialized successfully")
    print("✓ OpenAI client obtained")
    
except Exception as e:
    print(f"❌ Error initializing client: {e}")
    raise

✓ Azure AI Project client initialized successfully
✓ OpenAI client obtained


## 4. Chat Session Setup

Initialize the chat session with a system message and conversation history.

In [5]:
# Initialize conversation history
conversation_history = []

# Default system message
default_system_message = "You are a helpful AI assistant that answers questions."

def initialize_chat_session(system_message=None):
    """
    Initialize or reset the chat session with a system message.
    """
    global conversation_history
    
    if system_message is None:
        system_message = default_system_message
    
    conversation_history = [
        {"role": "system", "content": system_message}
    ]
    
    print(f"✓ Chat session initialized")
    print(f"System message: {system_message}")
    return conversation_history

# Initialize the chat session
initialize_chat_session()

✓ Chat session initialized
System message: You are a helpful AI assistant that answers questions.


[{'role': 'system',
  'content': 'You are a helpful AI assistant that answers questions.'}]

## 5. Chat Functions

Define helper functions for sending messages and managing the conversation.

In [6]:
def send_message(user_input, show_conversation=True):
    """
    Send a message to the AI and get a response.
    """
    global conversation_history
    
    if not user_input or len(user_input.strip()) == 0:
        print("Please enter a valid message.")
        return None
    
    try:
        # Add user message to conversation
        conversation_history.append({"role": "user", "content": user_input})
        
        # Get response from AI
        response = openai_client.chat.completions.create(
            model=model_deployment,
            messages=conversation_history
        )
        
        # Extract the completion
        completion = response.choices[0].message.content
        
        # Add assistant response to conversation
        conversation_history.append({"role": "assistant", "content": completion})
        
        if show_conversation:
            display(Markdown(f"**You:** {user_input}"))
            display(Markdown(f"**Assistant:** {completion}"))
            print("-" * 50)
        
        return completion
        
    except Exception as e:
        print(f"❌ Error sending message: {e}")
        return None

def show_conversation():
    """
    Display the current conversation history.
    """
    print(f"Conversation History ({len(conversation_history)} messages):")
    print("=" * 60)
    
    for i, message in enumerate(conversation_history):
        role = message["role"].title()
        content = message["content"]
        
        if role == "System":
            print(f"🔧 {role}: {content}")
        elif role == "User":
            print(f"👤 {role}: {content}")
        elif role == "Assistant":
            print(f"🤖 {role}: {content}")
        
        if i < len(conversation_history) - 1:
            print("-" * 40)
    
    print("=" * 60)

def clear_conversation():
    """
    Clear the conversation history but keep the system message.
    """
    initialize_chat_session()
    print("✓ Conversation history cleared")

print("✓ Chat functions defined")

✓ Chat functions defined


## 6. Interactive Chat Interface

Now you can start chatting! Use the cell below to send messages to the AI.

In [7]:
# Send a message to the AI
user_message = "Hello! Can you tell me about Azure AI Projects?"

response = send_message(user_message)
if response:
    print(f"\n✓ Message sent successfully. Response received.")

**You:** Hello! Can you tell me about Azure AI Projects?

**Assistant:** Certainly! Azure AI is a suite of AI services and tools provided by Microsoft Azure that allows developers and organizations to build, deploy, and manage AI models and applications. Here are some key components and types of projects you might encounter when working with Azure AI:

1. **Azure Machine Learning**:
   - A comprehensive machine learning platform that supports the end-to-end machine learning lifecycle.
   - Features include automated machine learning (AutoML), model deployment, and management tools.
   - You can build models using popular frameworks like TensorFlow, PyTorch, and scikit-learn.

2. **Cognitive Services**:
   - A set of pre-built APIs and services that allow developers to integrate AI capabilities into their applications easily.
   - Categories include Vision (computer vision, face recognition), Speech (speech-to-text, text-to-speech), Language (text analytics, translation, language understanding), and Decision (personalizer, anomaly detector).
   - Ideal for businesses looking to add AI features without developing models from scratch.

3. **Azure Bot Services**:
   - A platform for building intelligent bots that can interact with users across multiple channels (e.g., websites, Microsoft Teams, Slack).
   - Combines Azure Bot Framework with Cognitive Services to enable natural language understanding and dialogue management.
   
4. **Azure Databricks**:
   - An Apache Spark-based analytics platform optimized for Azure, useful for data engineering and data science projects.
   - Supports collaborative work environments for building and training AI models.

5. **Natural Language Processing (NLP) Projects**:
   - Projects that utilize Azure’s NLP capabilities for tasks such as sentiment analysis, entity recognition, and text generation.
   - Can be powered by services like Azure Language Understanding (LUIS) and Azure Text Analytics.

6. **Computer Vision Projects**:
   - These focus on analyzing and understanding images and videos using Azure's Computer Vision API.
   - Applications include image classification, object detection, and optical character recognition.

7. **Predictive Maintenance**:
   - Using Azure IoT and Machine Learning to predict equipment failures before they occur based on sensor data.
   - Enhances operational efficiency and reduces downtime in industrial settings.

8. **Recommendation Systems**:
   - Using Azure Personalizer or other machine learning techniques to build systems that provide personalized product or content recommendations based on user behavior.

9. **Custom AI Models**:
   - Organizations can build and train their own models using Azure Machine Learning with their unique datasets.
   - This includes deep learning, reinforcement learning, and other techniques tailored to specific business needs.

10. **Integration with Other Azure Services**:
    - Azure AI can be integrated with other Azure services such as Azure Functions, Azure Logic Apps, and Azure DevOps for more comprehensive solutions.

These projects can be applied across various industries, including healthcare (medical diagnosis), finance (fraud detection), retail (customer engagement), and more, showcasing the versatility of Azure AI in solving real-world problems.

Overall, Azure AI provides tools that cater to both novice developers and experienced data scientists, making it a robust platform for developing intelligent applications.

--------------------------------------------------

✓ Message sent successfully. Response received.


## 7. Custom Message Input

Modify the message below and run the cell to send your own questions.

In [8]:
# Customize this message and run the cell
your_message = "What are the key benefits of using Azure AI for building chat applications?"

response = send_message(your_message)
if response:
    print(f"\n✓ Message sent successfully.")

**You:** What are the key benefits of using Azure AI for building chat applications?

**Assistant:** Building chat applications using Azure AI offers several key benefits that can enhance the functionality, user experience, and operational efficiency of these applications. Here are some of the main advantages:

1. **Natural Language Understanding**:
   - **Language Understanding (LUIS)**: Azure’s Language Understanding Intelligent Service (LUIS) allows your chat application to comprehend user intents and extract entities from natural language input. This enables more accurate responses to user queries.

2. **Pre-built Models and Templates**:
   - Azure provides reusable models and templates for common use-cases, allowing developers to accelerate the development process without starting from scratch.

3. **Rich Conversational Experiences**:
   - Azure Bot Service provides the framework to build sophisticated bots that can manage multi-turn conversations, maintain context, and deliver dynamic responses.

4. **Integration with Various Channels**:
   - Azure Bot Service supports multiple communication channels, such as Microsoft Teams, Slack, Facebook Messenger, and web chat, making it easier to reach users where they are.

5. **Customization and Personalization**:
   - Utilizing Cognitive Services such as Personalizer allows chat applications to offer tailored experiences based on user behavior and preferences, enhancing user engagement.

6. **Speech Capabilities**:
   - Azure Speech Services enable voice recognition and text-to-speech, allowing chat applications to support voice interactions, making them more accessible and interactive for users.

7. **Scalability**:
   - As a cloud-based service, Azure AI offers scalability to handle varying loads. Your chat application can automatically scale to accommodate sudden increases in user interactions.

8. **In-depth Analytics**:
   - Azure provides tools for monitoring and analyzing user interactions, enabling developers to gain insights into user behavior, chat performance, and areas for improvement.

9. **Easy Deployment and Management**:
   - Azure simplifies the deployment and management of chat applications with built-in DevOps tools. You can version your bots, monitor performance, and roll out updates efficiently.

10. **Security and Compliance**:
    - Microsoft Azure adheres to strict security and compliance standards, ensuring that user data is protected. Features like authentication and authorization can be easily integrated into chat applications.

11. **Integration with Other Azure Services**:
    - Azure AI can seamlessly integrate with other Azure services such as Azure Functions for serverless components, Azure Logic Apps for workflow automation, and Azure Cosmos DB for scalable data storage.

12. **Machine Learning Capabilities**:
    - You can leverage Azure Machine Learning to create custom models tailored to specific business needs or use cases, enhancing the intelligence of the chat application.

13. **Cost-effective**:
    - Azure AI services generally follow a consumption-based pricing model, which can help control costs and ensure that you only pay for what you use, making it suitable for startups and enterprises alike.

By leveraging Azure AI for building chat applications, developers can create intelligent, responsive, and scalable solutions that enhance user interaction and satisfaction. These benefits combine to streamline development processes while producing a rich, engaging experience for end users.

--------------------------------------------------

✓ Message sent successfully.


## 8. Conversation Management

Use these cells to manage your conversation history.

In [9]:
# View the current conversation
show_conversation()

Conversation History (5 messages):
🔧 System: You are a helpful AI assistant that answers questions.
----------------------------------------
👤 User: Hello! Can you tell me about Azure AI Projects?
----------------------------------------
🤖 Assistant: Certainly! Azure AI is a suite of AI services and tools provided by Microsoft Azure that allows developers and organizations to build, deploy, and manage AI models and applications. Here are some key components and types of projects you might encounter when working with Azure AI:

1. **Azure Machine Learning**:
   - A comprehensive machine learning platform that supports the end-to-end machine learning lifecycle.
   - Features include automated machine learning (AutoML), model deployment, and management tools.
   - You can build models using popular frameworks like TensorFlow, PyTorch, and scikit-learn.

2. **Cognitive Services**:
   - A set of pre-built APIs and services that allow developers to integrate AI capabilities into their applic

In [ ]:
# Clear conversation history
clear_conversation()

## 9. Advanced: Custom System Messages

Experiment with different system messages to change the AI's behavior.

In [ ]:
# Try different system messages
custom_system_messages = {
    "helpful": "You are a helpful AI assistant that answers questions.",
    "creative": "You are a creative AI assistant that provides imaginative and innovative responses.",
    "technical": "You are a technical AI assistant specialized in software development and Azure technologies.",
    "educator": "You are an AI tutor that explains concepts clearly and asks follow-up questions to ensure understanding.",
    "casual": "You are a friendly, casual AI assistant that responds in a conversational tone."
}

# Choose a system message type
message_type = "technical"  # Change this to: helpful, creative, technical, educator, or casual

if message_type in custom_system_messages:
    initialize_chat_session(custom_system_messages[message_type])
    print(f"✓ System message set to '{message_type}' mode")
else:
    print(f"❌ Unknown message type. Available types: {', '.join(custom_system_messages.keys())}")

In [ ]:
# Test the new system message
test_message = "Explain how to deploy an Azure Function."

response = send_message(test_message)
if response:
    print(f"\n✓ Response generated with current system message.")

## 10. Conversation Export

Save your conversation history for later reference.

In [ ]:
def export_conversation(filename=None):
    """
    Export the conversation history to a JSON file.
    """
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"chat_conversation_{timestamp}.json"
    
    conversation_data = {
        "timestamp": datetime.now().isoformat(),
        "model_deployment": model_deployment,
        "conversation": conversation_history
    }
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(conversation_data, f, indent=2, ensure_ascii=False)
        
        print(f"✓ Conversation exported to: {filename}")
        return filename
    
    except Exception as e:
        print(f"❌ Error exporting conversation: {e}")
        return None

# Export the current conversation
export_file = export_conversation()
print(f"Conversation contains {len(conversation_history)} messages")

## 11. Interactive Chat Loop (Optional)

Run this cell for a continuous chat experience similar to the original script.

In [ ]:
def interactive_chat_loop():
    """
    Run an interactive chat loop in the notebook.
    Note: This works best in Jupyter environments that support input().
    """
    print("🚀 Starting interactive chat session...")
    print("Type 'quit' to exit, 'clear' to clear history, 'show' to display conversation")
    print("-" * 60)
    
    while True:
        try:
            user_input = input("\n👤 You: ").strip()
            
            if user_input.lower() == 'quit':
                print("👋 Goodbye!")
                break
            elif user_input.lower() == 'clear':
                clear_conversation()
                continue
            elif user_input.lower() == 'show':
                show_conversation()
                continue
            elif len(user_input) == 0:
                print("Please enter a message.")
                continue
            
            # Send message and get response
            response = send_message(user_input, show_conversation=False)
            if response:
                print(f"\n🤖 Assistant: {response}")
            
        except KeyboardInterrupt:
            print("\n\n👋 Chat session interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

# Uncomment the line below to start the interactive chat loop
interactive_chat_loop()

print("Interactive chat loop function is ready. Uncomment the last line to start chatting!")

🚀 Starting interactive chat session...
Type 'quit' to exit, 'clear' to clear history, 'show' to display conversation
------------------------------------------------------------

🤖 Assistant: Hadoop is an open-source framework designed for distributed storage and processing of large data sets across clusters of computers using simple programming models. It is known for its ability to scale up from a single server to thousands of machines, each offering local computation and storage. Here are some key components and features of Hadoop:

### Key Components of Hadoop

1. **Hadoop Distributed File System (HDFS)**:
   - HDFS is the storage layer of Hadoop. It divides large data files into smaller blocks (typically 128 MB or 256 MB) and distributes them across the cluster.
   - It provides high-throughput access to application data and is designed to be fault-tolerant by replicating blocks across multiple machines.

2. **YARN (Yet Another Resource Negotiator)**:
   - YARN is the resource man

## Summary

This notebook provides a comprehensive interactive chat application with the following features:

✅ **Modular Design**: Each functionality is separated into different cells for easy experimentation

✅ **Environment Setup**: Automatic package installation and environment validation

✅ **Azure AI Integration**: Full integration with Azure AI Projects and OpenAI

✅ **Conversation Management**: View, clear, and export conversation history

✅ **Custom System Messages**: Experiment with different AI personalities

✅ **Interactive Features**: Both cell-by-cell and continuous chat modes

✅ **Error Handling**: Comprehensive error handling and validation

### Next Steps:
- Modify system messages to create specialized AI assistants
- Experiment with different conversation flows
- Add more advanced features like conversation analysis
- Integrate with other Azure AI services

### Tips for Usage:
1. Run cells sequentially for the first time
2. Use the conversation management cells to track your chat history
3. Experiment with different system messages to see how they affect responses
4. Export important conversations for future reference